In [1]:
LABELD_DIR = "C:\\Users\\karam\\PycharmProjects\\skillExtraction\\labeled_jobpostings"
LABELD_DIR = "C:\\Users\\karam\\PycharmProjects\\GermanJobSkillExtractor\\tagedFiles"

In [2]:
from pathlib import Path

DOCS = [
    
    [
        line.strip()
        for line in file.read_text(encoding="utf8").split("\n")
        if line
    ]
    
    for file in Path(LABELD_DIR).glob("*.txt")
]

"No of docs:", len(DOCS)

('No of docs:', 137)

In [3]:
#from somajo import SoMaJo

#tokenizer = SoMaJo("de_CMC", split_camel_case=False)
from transformers import  AutoTokenizer, BertTokenizerFast
MODEL_NAME_OR_PATH = "C:\\Users\\karam\\Documents\\Job\\models\\agnejob__BERT-de"  # "agne/jobBERT-de")
MODEL_NAME_OR_PATH = "agne/jobGBERT"
TOKENIZER = BertTokenizerFast.from_pretrained(MODEL_NAME_OR_PATH)

In [4]:
TOKENIZER.tokenize("bei Intermarket Bank AG")

['bei', 'Inter', '##marke', '##t', 'Bank', 'AG']

In [5]:
import re
sent = "gjährige Erfahrung im Bereich <<*>>SQL<<*>>, <<*>>vorzugsweise Microsoft SQL<<*>> oder als <<*>>Business Analyst<<*>> bzw. <<*>>Business Consultant<<*>> im regulatorischen Umfeld<<*>>"
list(re.split(r"((?:<<\*>>).+?(?:<<\*>>))", sent))

['gjährige Erfahrung im Bereich ',
 '<<*>>SQL<<*>>',
 ', ',
 '<<*>>vorzugsweise Microsoft SQL<<*>>',
 ' oder als ',
 '<<*>>Business Analyst<<*>>',
 ' bzw. ',
 '<<*>>Business Consultant<<*>>',
 ' im regulatorischen Umfeld<<*>>']

In [6]:
import re
import json
from itertools import chain, repeat
from operator import itemgetter

SPLIT_TOK = r"<<\*>>"

def tag_sent_tokens(sent: str):
    sent_parts = list()
    parts = re.split(r"((?:<<\*>>).+?(?:<<\*>>))", sent, flags=re.IGNORECASE | re.UNICODE)
    #for part, pre in zip(parts, chain([None], parts)):
    #    print(pre, ">>", part)
    for part in parts:
        is_tagged = part.startswith("<<*>>")
        part = part.replace("<<*>>", "").strip()
        tokens = TOKENIZER.tokenize(part)
        if is_tagged:
            sent_parts.extend(t for t in zip(tokens, chain(["B"], repeat("I"))))
        else:
            sent_parts.extend((t, "O") for t in tokens)
        
    # print(sent_parts)

    return sent_parts

with open("./train_jp.json", "wt", encoding="utf8") as f:
    for doc in DOCS:
        for sent in doc:
            sent_parts = tag_sent_tokens(sent)
            if len(sent_parts) < 2: continue
            tokens = map(itemgetter(0), sent_parts)
            tags = map(itemgetter(1), sent_parts)
            json.dump(dict(tokens=list(tokens), tags=list(tags)), f, ensure_ascii=False, sort_keys=False)
            f.write("\n")

